In [2]:
!pip install llama-index
!pip install llama-index-multi-modal-llms-gemini
!pip install llama-index-embeddings-gemini

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.3/251.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [1]:
from getpass import getpass

GOOGLE_API_KEY = getpass("Enter your Google API Key: ")


Enter your Google API Key: ··········


In [3]:
import os
from llama_index.core import SimpleDirectoryReader, Settings

# Ensure the data folder exists
os.makedirs('data', exist_ok=True)

# Move the uploaded PDF to the 'data' directory if it's in the root
!mv HRD_Material.pdf data/

# Load the data from the PDF
documents = SimpleDirectoryReader('data').load_data()

# Create nodes from the document
nodes = Settings.node_parser.get_nodes_from_documents(documents)

# Preview the first 5 nodes
for node in nodes[:5]:
    print(node)




mv: cannot stat 'HRD_Material.pdf': No such file or directory


Node ID: 819eb40e-3fa2-413a-8380-461d8730a620
Text: A Simplified Guide To Fingerprint Analysis
Node ID: 863ff14d-a2d3-4cf4-a5fe-f9e162ed3c18
Text: Introduction We      touch    things           every    day:
 a        coffee           cup,     a        car      door,    a
 computer         keyboard.        Each     time     we       do,
 it       is       likely           that     we       leave    behind
 our      unique           signature—in     our      fingerprints.   
 No       two      people           have     exactly          the
 same     fingerprints.   ...
Node ID: 5001abb5-40cf-403a-bcaa-93476eb960a0
Text: Principles of Fingerprint Analysis Fingerprints      are
 unique           patterns,        made     by       friction
 ridges           (raised)         and      furrows
 (recessed),      which    appear           on       the      pads
 of       the      fingers          and      thumbs.          Prints
 from     palms,           toes     and      feet     are      als

In [5]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

# Set up the embedding model and LLM
Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001", api_key=GOOGLE_API_KEY)
Settings.llm = Gemini(api_key=GOOGLE_API_KEY)


<ipython-input-5-c99597005e7f>:5: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001", api_key=GOOGLE_API_KEY)
<ipython-input-5-c99597005e7f>:6: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  Settings.llm = Gemini(api_key=GOOGLE_API_KEY)


In [6]:
from llama_index.core import StorageContext

# Create a default storage context and add nodes
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)


In [7]:
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex

# Creating vector and keyword indices
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

print("VectorStoreIndex and SimpleKeywordTableIndex created successfully!")


VectorStoreIndex and SimpleKeywordTableIndex created successfully!


In [8]:
from llama_index.core import QueryBundle
from llama_index.core.schema import NodeWithScore
from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever, KeywordTableSimpleRetriever
from typing import List

class CustomRetriever(BaseRetriever):
    def __init__(self, vector_retriever, keyword_retriever, mode="AND"):
        self._vector_retriever = vector_retriever
        self._keyword_retriever = keyword_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode. Use 'AND' or 'OR'.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        keyword_nodes = self._keyword_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        keyword_ids = {n.node.node_id for n in keyword_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in keyword_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(keyword_ids)
        else:
            retrieve_ids = vector_ids.union(keyword_ids)

        return [combined_dict[r_id] for r_id in retrieve_ids]


In [9]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

# Adjusted similarity_top_k to 5 for better retrieval
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=5)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)

# Create a custom retriever
custom_retriever = CustomRetriever(vector_retriever, keyword_retriever)

# Create a response synthesizer
response_synthesizer = get_response_synthesizer()

# Create the custom query engine
custom_query_engine = RetrieverQueryEngine(retriever=custom_retriever, response_synthesizer=response_synthesizer)

# Test queries
print(custom_query_engine.query("What does the data context contain?"))
print(custom_query_engine.query("Can you summarize the content of this material?"))


The data context includes a simplified guide to fingerprint analysis and information on fingerprint analysis principles. It also mentions when the opponent can question potential witnesses.

This document provides a simplified guide to fingerprint analysis, focusing on prints from fingers and thumbs. It states that fingerprints are unique patterns, made by friction ridges (raised) and furrows (recessed) that appear on the pads of the fingers and thumbs. It also mentions that during court cases, the opponent can question potential witnesses in an attempt to show that they do not have applicable expertise and are not qualified to testify on the topic, with the admissibility decision being left to the judge. Concerns over evidence can be handled through cross-examination, presentation of contrary evidence, and instruction on the burden of proof.

